<a href="https://colab.research.google.com/github/jlopetegui98/Creation-of-a-synthetic-dataset-for-French-NER-in-clinical-trial-texts/blob/main/NER-chia-dataset/model_ner_chia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Multilingual NER model trained over [Chia dataset](https://figshare.com/articles/dataset/Chia_Annotated_Datasets/11855817)**

We are going to train a BERT based multilingual language model over the Chia dataset in english and then we will use this model to create the synthetic version of the dataset in French. Our idea is supported by the experiments already done with [multiNERD](https://huggingface.co/datasets/Babelscape/multinerd) dataset for multilingual NER in English and French.

**Entities selection**

Among all the entities in the dataset, we are going to focus for this project on the most represented ones. Then, we are just going to consider those entities with more than 1000 samples in total.

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f2d7ae1e20cc9aa4312b4b063423038906980abd54b773f7641793507f0351a1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
# imports
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import os
# from preprocessing_dataset import *
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
import json
from datasets.features import ClassLabel

In [4]:
# dict for the entities (entity to int value)
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Grug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16
}
entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [5]:
# data paths
root_path = './' # comment if working on colab
root_path = './drive/MyDrive/HandsOn-NLP'
data_path = f'{root_path}/data'
chia_bio_path = f"{data_path}/chia_bio"
chia_prep_path = f"{data_path}/chia_prep"
models_path = f"{root_path}/models"

In [6]:
# preprocessing dataset to get the data in the right format for dataset entity creation
preprocessing_dataset(chia_bio_path, output_path=chia_prep_path, labels2int = sel_ent, int2labels = sel_ent_inv)

NameError: name 'preprocessing_dataset' is not defined

In [7]:
# read the data after preprocessing
files = os.listdir(chia_prep_path)
files[:5]

['NCT02893293_exc.bio.json',
 'NCT03475589_exc.bio.json',
 'NCT01997580_inc.bio.json',
 'NCT01993836_exc.bio.json',
 'NCT02589691_inc.bio.json']

In [8]:
sentences = []

for file in files:
    with open(f"{chia_prep_path}/{file}", "r") as f:
        stc = json.load(f)
        sentences.extend(stc["sentences"])

In [9]:
# create the dataset
chia_eng_dataset = Dataset.from_pandas(pd.DataFrame(sentences))

In [10]:
chia_eng_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 12423
})

In [11]:
chia_eng_train_test = chia_eng_dataset.train_test_split(test_size=0.2)
chia_eng_test_val = chia_eng_train_test["test"].train_test_split(test_size=0.5)
chia_eng_dataset = DatasetDict({
    "train": chia_eng_train_test["train"],
    "test": chia_eng_test_val["test"],
    "validation": chia_eng_test_val["train"]
})

In [12]:
chia_eng_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9938
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1243
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1242
    })
})

**Model Implementation**

In [13]:
model_name = 'xlm-roberta-base'

In [14]:
# get xlm-roberta tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# check the tokenizer
tokens_ = tokenizer("The AI master at Université Paris-Saclay is very good").tokens()
print(tokens_)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['<s>', '▁The', '▁AI', '▁master', '▁at', '▁', 'Université', '▁Paris', '-', 'S', 'ac', 'lay', '▁is', '▁very', '▁good', '</s>']


In [15]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

In [17]:
type(chia_eng_dataset)

datasets.dataset_dict.DatasetDict

In [18]:
# apply the function to the dataset
chia_eng_dataset = chia_eng_dataset.map(tokenize_and_align_labels, batched=True)
chia_eng_dataset

Map:   0%|          | 0/9938 [00:00<?, ? examples/s]

Map:   0%|          | 0/1243 [00:00<?, ? examples/s]

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9938
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1243
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1242
    })
})

In [19]:
# import the model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(entities_list), label2id=sel_ent, id2label=sel_ent_inv)
print(model)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
model.to(device)

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [22]:
# define the training arguments
args = TrainingArguments(
    "chia-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
)

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
metric = load_metric("seqeval")

<ipython-input-27-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [30]:
def compute_metrics(p):
    """
    Compute the metrics for the model
    inputs:
        p: tuple, the predictions and the labels
    outputs:
        dict: the metrics
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, labels=np.unique(true_predictions))
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [31]:
# define the trainer
trainer = Trainer(
    model,
    args,
    train_dataset=chia_eng_dataset["train"],
    eval_dataset=chia_eng_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [32]:
outputs_train = trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,No log,0.979678,0.561404,0.559010,0.560204,0.725094
100,No log,0.819582,0.636015,0.638377,0.637194,0.763851
150,No log,0.753130,0.646868,0.657634,0.652207,0.782574
200,No log,0.689003,0.675120,0.677442,0.676279,0.790150
250,No log,0.659232,0.674771,0.679505,0.677130,0.792445
300,No log,0.640743,0.701863,0.714993,0.708367,0.802681
350,No log,0.633061,0.704628,0.718294,0.711396,0.802900
400,No log,0.611747,0.708831,0.750757,0.729192,0.813499
450,No log,0.615388,0.702999,0.741678,0.721821,0.808145
500,0.762300,0.589135,0.704187,0.749519,0.726146,0.814119


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

In [33]:
print(outputs_train)

TrainOutput(global_step=1866, training_loss=0.548830651776967, metrics={'train_runtime': 2000.7226, 'train_samples_per_second': 14.902, 'train_steps_per_second': 0.933, 'total_flos': 1177014673199124.0, 'train_loss': 0.548830651776967, 'epoch': 3.0})


In [34]:
outputs_eval = trainer.evaluate(chia_eng_dataset["test"])

In [35]:
print(outputs_eval)

{'eval_loss': 0.5564100742340088, 'eval_precision': 0.7481307117142066, 'eval_recall': 0.7751793400286944, 'eval_f1': 0.7614148816234498, 'eval_accuracy': 0.830517690007379, 'eval_runtime': 5.5034, 'eval_samples_per_second': 225.862, 'eval_steps_per_second': 28.346, 'epoch': 3.0}


In [36]:
torch.save(model, f"{models_path}/chia-multilingual-ner.pt")